In [10]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [11]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]

display(df_refugee.head())

,Year,Country / territory of asylum/residence,Origin,Population type,Value
0,1951,Australia,Various/Unknown,Refugees (incl. refugee-like situations),180000.0
1,1951,Austria,Various/Unknown,Refugees (incl. refugee-like situations),282000.0
2,1951,Belgium,Various/Unknown,Refugees (incl. refugee-like situations),55000.0
3,1951,Canada,Various/Unknown,Refugees (incl. refugee-like situations),168511.0
4,1951,Switzerland,Various/Unknown,Refugees (incl. refugee-like situations),10000.0


In [12]:
df_geo  = pd.read_csv("geo.csv")
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['Year','Value','Origin',"country"]]
df_merge = df_merge.rename(columns={"country" : "country_asylum"})
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
df_merge = df_merge[['Year',"country","country_asylum",'Value']]
df_merge = df_merge.rename(columns={"country":"country_origin"})
df_merge = df_merge.sort_values("Year")

display(df_merge.head())

,Year,country_origin,country_asylum,Value
72887,1961,RW,BI,30000.0
73120,1961,RW,UG,52000.0
35836,1961,GH,TG,5000.0
73049,1961,RW,TZ,12000.0
72888,1962,RW,BI,30000.0


In [13]:
df_immigration_exit = df_merge.drop("country_asylum",1)
df_immigration_exit = df_immigration_exit.groupby(["Year","country_origin"]).sum().reset_index()
df_immigration_exit= df_immigration_exit.sort_values(["country_origin","Year"])
display(df_immigration_exit.tail())
df_immigration_exit.to_csv("data_immigration_exit.csv", sep=',', encoding='utf-8',index = False)

,Year,country_origin,Value
4732,2012,ZW,121800.0
4916,2013,ZW,121810.0
5101,2014,ZW,65097.0
5287,2015,ZW,78899.0
5451,2016,ZW,61458.0


In [14]:
df_immigration_entry = df_merge.drop("country_origin",1)
df_immigration_entry = df_immigration_entry.groupby(["Year","country_asylum"]).sum().reset_index()
df_immigration_entry= df_immigration_entry.sort_values(["country_asylum","Year"])
df_immigration_entry = df_immigration_entry[["country_asylum","Year","Value"]]
display(df_immigration_entry.tail())
df_immigration_entry.to_csv("data_immigration_entry.csv", sep=',', encoding='utf-8',index = False)

,country_asylum,Year,Value
3976,ZW,2012,59376.0
4139,ZW,2013,61922.0
4301,ZW,2014,1704.0
4464,ZW,2015,2166.0
4612,ZW,2016,2500.0
